In [1]:
# 라이브러리 설치 및 호출
!pip install ipywidgets pdfplumber openai==0.28 konlpy

import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import warnings
from google.colab import output, files
import openai
import pdfplumber
from konlpy.tag import Okt
import re
from collections import Counter

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 58.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 53.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 496.6/496.6 kB 29.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 62.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 58.2 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.99.8
    Uninstalling openai-1.99.8:
      Successfully uninstalled openai-1.99.8


In [ ]:
# 경고 메시지 필터링
warnings.filterwarnings("ignore")

# Colab 위젯 매니저 활성화 (UI가 제대로 표시되도록)
output.enable_custom_widget_manager()

# API 키 설정
openai.api_key = "본인이 발급 받은 key"

In [ ]:
# KoNLPy 한국어 자연어처리 파이썬 패키지를 사용한 텍스트 전처리
okt = Okt() # 객체 초기화

# 불용어 리스트 생성
korean_stopwords = [
    '은', '는', '이', '가', '을', '를', '에', '에서', '에게', '에게서', '으로', '로', '와', '과',
    '도', '만', '요', '다', '습니다', 'ㅂ니다', '습니다', '있다', '합니다', '것', '수', '같다', '있다',
    '처럼', '듯이', '위해', '통해', '등', '이다', '아니다', '말하다', '알다', '되다', '지다', '보다',
    '때', '로', '에', '에서', '하다', '때문', '로부터', '하는', '될', '그', '저', '수', '또', '안',
    '그것', '이것', '저것', '하나', '둘', '셋', '넷', '다섯', '여섯', '일곱', '여덟', '아홉', '열',
    '저희', '우리', '있는', '있었다', '이렇다', '그렇다', '저렇다', '않다', '없다'
]

# 불용어 제거 함수 정의
def preprocess_text_only_stopwords_removal(input_text):
    # input_text가 문자열이 아닐 경우 문자열로 강제 변환
    if not isinstance(input_text, str):
        if isinstance(input_text, list):
            input_text = ' '.join(input_text)
        else:
            input_text = str(input_text)

    words = []
    tagged_words = okt.pos(input_text, norm=True, stem=True)

    for word, tag in tagged_words:
        # 명사(Noun), 동사(Verb), 형용사(Adjective)만 선택
        if tag in ['Noun', 'Verb', 'Adjective']:
            # 불용어 아니고 두 글자 이상일 경우
            if word not in korean_stopwords and len(word) > 1:
                words.append(word)

    return ' '.join(words) # 전처리된 단어들을 공백으로 연결한 문자열 반환

# 키워드 추출 함수 정의 (Term Frequency 계산용)
def extract_keywords_with_tf(input_text_string):
    processed_words_string = preprocess_text_only_stopwords_removal(input_text_string)
    return Counter(processed_words_string.split())

In [ ]:
# PDF 업로드 및 각 파일별 텍스트 추출 & 전처리 실행
print("서평 PDF 파일들을 모두 업로드해주세요.")
uploaded = files.upload()

if not uploaded:
    print("업로드된 파일이 없습니다. 스크립트를 종료합니다.")
    raise SystemExit("No files uploaded.")

# 전역 변수 선언 및 초기화 (모든 함수에서 접근할 수 있도록 global로 선언)
global processed_document_chunks
global document_titles

processed_document_chunks = []
document_titles = []

# 업로드된 모든 파일 이름을 순회
for pdf_file_name in uploaded.keys():
    print(f"--- Processing file: {pdf_file_name} ---")
    current_pdf_raw_text = ""
    try:
        with pdfplumber.open(pdf_file_name) as pdf:
            for page in pdf.pages:
                page_text = page.extract_text()
                if page_text:
                    current_pdf_raw_text += page_text + "\n"
    except Exception as e:
        print(f"An error occurred while extracting text from '{pdf_file_name}': {e}")
        continue

    preprocessed_doc_text = preprocess_text_only_stopwords_removal(current_pdf_raw_text)

    processed_document_chunks.append(preprocessed_doc_text)
    document_titles.append(pdf_file_name)

    print(f"'{pdf_file_name}' 전처리 후 글자 수: {len(preprocessed_doc_text)}")

print("\n모든 서평 PDF 파일 전처리 완료!")

# 시스템이 인식하는 책 제목 확인 (디버깅용)
print("\n\n시스템이 인식하는 책 제목 목록")
for i, title in enumerate(document_titles):
    clean_title = title.replace('.pdf', '').strip().lower()
    print(f"원본 파일명: '{title}' -> 시스템 인식 제목: '{clean_title}'")

In [ ]:
# 이전 대화 기록(로그) 저장 변수 초기화
initial_system_message_content = (
    "당신은 책 소개 문서를 이해하고 응답하는 챗봇입니다. "
    f"다음 책들에 대해 질문할 수 있습니다: {', '.join([title.replace('.pdf', '') for title in document_titles])}. "
    "질문 시에는 어떤 책에 대한 질문인지 명시해 주시면 더 정확한 답변을 드릴 수 있습니다."
    "예시 질문: '사랑의 기술을 요약해줘.', '사랑의 기술의 비통제 색인어를 만들어줘', '모든 책들의 키워드를 알려줘'"
)

# 대화 기록 저장 리스트
conversation_history = [
    {"role": "system", "content": initial_system_message_content}
]

In [ ]:
# ChatGPT API 호출 함수 정의 (대화 기록 반영 ver)
def ask_chatgpt_with_history(user_question):
    global conversation_history
    global processed_document_chunks
    global document_titles

    selected_doc_text_for_prompt = ""
    selected_doc_title_for_prompt = "여러 문서 요약"

    found_relevant_doc = False
    for i, title in enumerate(document_titles):
        clean_title = title.replace('.pdf', '').strip().lower()
        if clean_title in user_question.lower():
            selected_doc_text_for_prompt = processed_document_chunks[i]
            selected_doc_title_for_prompt = title
            found_relevant_doc = True
            break

    if not found_relevant_doc:
        combined_docs_preview = []
        for i, doc_text in enumerate(processed_document_chunks):
            combined_docs_preview.append(f"[{document_titles[i].replace('.pdf', '')}]: {doc_text[:300]}...")

        selected_doc_text_for_prompt = "\n\n".join(combined_docs_preview)

    effective_document_for_prompt = selected_doc_text_for_prompt[:3000]

    new_system_message = {
        "role": "system",
        "content": (
            f"당신은 책 소개 문서를 이해하고 응답하는 챗봇입니다. 다음 문서 내용을 바탕으로 질문에 답변하세요:\n\n"
            f"참고 문서 내용 ({selected_doc_title_for_prompt.replace('.pdf', '')})\n"
            f"{effective_document_for_prompt}\n"
            f"--------------------------------------------------\n"
            f"만약 위 문서 내용에 해당 정보가 명확히 없으면, '문서에서 해당 정보를 찾을 수 없습니다.'라고 간결하게 답변해주세요."
        )
    }

    current_dialog_turns_only = conversation_history[1:]

    # 최대 대화 기록 반영 수 5개로 설정
    MAX_CONVERSATION_TURNS = 5
    if len(current_dialog_turns_only) > (MAX_CONVERSATION_TURNS * 2):
        current_dialog_turns_only = current_dialog_turns_only[-(MAX_CONVERSATION_TURNS * 2):]

    messages_to_model = [new_system_message] + current_dialog_turns_only

    try:
        messages_to_model.append({"role": "user", "content": user_question})

        response = openai.ChatCompletion.create(
            model="gpt-4o",
            messages=messages_to_model,
            temperature=0.2, # 사실 기반 답변을 위해 temperature 매개변수 값을 0.2로 설정
            max_tokens=500
        )
        if not response.choices or not response.choices[0].message["content"]:
            raise ValueError("The model responded with an empty message.")

        chatbot_response_content = response.choices[0].message["content"]

        conversation_history.append({"role": "user", "content": user_question})
        conversation_history.append({"role": "assistant", "content": chatbot_response_content})

        return chatbot_response_content

    except Exception as e:
        if conversation_history and conversation_history[-1]["role"] == "user":
            conversation_history.pop()
        return f"죄송합니다, 답변을 생성하는 중에 오류가 발생했습니다: {e}"

In [ ]:
# 위젯 구성
chat_display = widgets.HTML(value="<b>📄 책봇에게 질문해 보세요!</b><hr>")
chat_log = ""

input_box = widgets.Textarea(
    placeholder='질문을 입력하세요 (예: 사랑의 기술의 줄거리는? / 사랑의 기술의 비통제 색인어 만들어줘 / 모든 책 키워드 알려줘)',
    layout=widgets.Layout(width='100%', height='80px')
)

send_button = widgets.Button(description="💬 질문하기", button_style='info')

# 버튼 동작 정의
def on_send_clicked(b):
    global chat_log
    user_input = input_box.value.strip()
    if not user_input:
        return
    input_box.value = ""

    clear_output(wait=True)
    chat_log += f"<div style='margin-bottom:10px'><b>🙋 사용자:</b> {user_input}</div>"

    current_chat_display_value = chat_log + "<div><i>🤖 ChatGPT 입력 중...</i></div>"
    chat_display.value = current_chat_display_value
    display(chat_display, input_box, send_button)

    answer = ""
    try:
        user_input_lower = user_input.lower()
        user_input_no_spaces = user_input_lower.replace(" ", "") # 사용자 입력에서 모든 공백 제거

        # KORMARC 520 태그 기능
        kormarc_520_trigger_terms = ["520 태그", "520 tag", "요약 태그", "요약 주기", "520 marc", "520 MARC", "520 마크"]
        if any(term in user_input_lower for term in kormarc_520_trigger_terms):
            selected_doc_text_for_summary = ""
            selected_doc_title_for_summary = "모든 문서 요약"

            found_specific_book_for_summary = False
            for i, title in enumerate(document_titles):
                # 문서 제목도 비교를 위해 공백 제거
                clean_title_no_spaces = title.replace('.pdf', '').strip().lower().replace(" ", "")
                # 공백 제거된 사용자 입력과 공백 제거된 문서 제목 비교
                if clean_title_no_spaces in user_input_no_spaces:
                    selected_doc_text_for_summary = processed_document_chunks[i]
                    selected_doc_title_for_summary = title
                    found_specific_book_for_summary = True
                    break

            if not found_specific_book_for_summary:
                combined_docs_preview_summary = []
                for i, doc_text in enumerate(processed_document_chunks):
                    combined_docs_preview_summary.append(f"[{document_titles[i].replace('.pdf', '')}]: {doc_text[:300]}...")
                selected_doc_text_for_summary = "\n\n".join(combined_docs_preview_summary)

            kormarc_520_prompt = (
                f"다음 책 소개 문서의 내용을 바탕으로 KORMARC 520 태그(요약)를 작성해주세요. "
                f"반드시 '520 ## $a ' 형식으로 시작하고, 뒤에 요약 텍스트를 붙여주세요. "
                f"요약 텍스트는 간결하고 핵심적인 내용으로 100자 이내로 작성해주세요. "
                f"다른 설명이나 문장 없이 KORMARC 태그 형식만 출력하세요."
                f"문서 내용: {selected_doc_text_for_summary[:3000]}"
            )

            generated_kormarc_520 = ask_chatgpt_with_history(kormarc_520_prompt)

            answer = f"🤖 책봇 제안 KORMARC 520 태그 ({selected_doc_title_for_summary.replace('.pdf', '')} 참고):\n"
            answer += f"```\n{generated_kormarc_520}\n```\n"

        # KORMARC 653 태그 생성 및 최빈 키워드 제시 기능
        kormarc_653_trigger_terms = ["비통제 색인어", "kormarc 653", "653 tag", "653 marc", "핵심 단어", "핵심 키워드", "핵심어", "색인어", "653 태그", "653 마크"]
        if any(term in user_input_lower for term in kormarc_653_trigger_terms):
            selected_doc_text_for_kormarc = ""
            selected_doc_title_for_kormarc = "모든 문서 요약"
            found_specific_book_for_kormarc = False

            # 각 문서 제목을 순회하며 사용자 입력과 매칭 시도
            for i, title in enumerate(document_titles):
                clean_title_for_comparison = title.replace('.pdf', '').strip().lower()
                if clean_title_for_comparison in user_input_lower:
                    selected_doc_text_for_kormarc = processed_document_chunks[i]
                    selected_doc_title_for_kormarc = title
                    found_specific_book_for_kormarc = True
                    break

            if not found_specific_book_for_kormarc:
                combined_docs_preview_kormarc = []
                for i, doc_text in enumerate(processed_document_chunks):
                    combined_docs_preview_kormarc.append(f"[{document_titles[i].replace('.pdf', '')}]: {doc_text[:300]}...")
                selected_doc_text_for_kormarc = "\n\n".join(combined_docs_preview_kormarc)

            kormarc_prompt = (
                f"다음 책 소개 문서의 내용을 바탕으로 KORMARC 653 태그(비통제 색인어)에 해당하는 핵심 키워드 5개를 생성해주세요. "
                f"예시: '$a독서 심리 $a관계 형성 $a감성 지능 $a독서 클럽 $a도서관 이용' 과 같이 KORMARC 형식에 맞춰 출력해주세요. "
                f"추가 설명 없이 색인어만 출력하세요."
                f"문서 내용: {selected_doc_text_for_kormarc[:3000]}\n"
                f"반드시 '653 ## $a ' 형식으로 작성해주세요."
            )

            generated_kormarc_terms = ask_chatgpt_with_history(kormarc_prompt)

            if found_specific_book_for_kormarc:
                keywords_for_tf = extract_keywords_with_tf(selected_doc_text_for_kormarc)
                current_book_title = selected_doc_title_for_kormarc.replace('.pdf', '')
            else:
                keywords_for_tf = extract_keywords_with_tf(" ".join(processed_document_chunks))
                current_book_title = "모든 문서"

            top_3_keywords = keywords_for_tf.most_common(3)

            answer = f"🤖 책봇 제안 비통제 색인어 ({current_book_title} 참고):\n"
            answer += f"```\n{generated_kormarc_terms}\n```\n\n"
            answer += "<br>💡 참고: 문서에서 가장 많이 출현한 키워드 (TF 분석 결과)\n"
            if top_3_keywords:
                for keyword, count in top_3_keywords:
                    answer += f"<br>⚠️ {keyword} (등장 횟수: {count}회)<br>"
            else:
                answer += "키워드를 찾을 수 없습니다.<br>"

        # 일반 키워드 추출 로직
        elif "키워드" in user_input_lower or "핵심어" in user_input_lower:
            relevant_doc_text_for_keywords = ""
            current_book_title_for_keywords = "모든 문서"
            found_specific_book_for_keywords = False

            # 각 문서 제목을 순회하며 사용자 입력과 매칭 시도 (동일한 로직)
            for i, title in enumerate(document_titles):
                clean_title_for_comparison = title.replace('.pdf', '').strip().lower()
                if clean_title_for_comparison in user_input_lower:
                    relevant_doc_text_for_keywords = processed_document_chunks[i]
                    current_book_title_for_keywords = title.replace('.pdf', '')
                    found_specific_book_for_keywords = True
                    break

            if not found_specific_book_for_keywords:
                combined_all_text_for_keywords = " ".join(processed_document_chunks)
                keywords_counter = extract_keywords_with_tf(combined_all_text_for_keywords)
            else:
                keywords_counter = extract_keywords_with_tf(relevant_doc_text_for_keywords)

            top_keywords_list = [word for word, count in keywords_counter.most_common(20)]
            answer = f"({current_book_title_for_keywords} 참고) 추출된 주요 키워드는 다음과 같습니다: {', '.join(top_keywords_list)}"

        else:
            answer = ask_chatgpt_with_history(user_input)

    except Exception as e:
        answer = f"죄송합니다. 요청을 처리하는 중에 오류가 발생했습니다: {e}"

    chat_log += f"<div style='margin-bottom:20px'><b>🤖 ChatGPT:</b> {answer}</div>"
    chat_display.value = chat_log

# 버튼 클릭 이벤트 연결
send_button.on_click(on_send_clicked)

# 챗봇 UI 출력
display(chat_display)
display(input_box, send_button)